In [1]:
import tensorflow as tf
import pandas as pd
import pickle
tf.compat.v1.disable_eager_execution()

In [2]:
df = pd.read_json('Spotify_Lyrics_B_trimmed.json')
with open('Spotify_Embedding_B.pkl', 'rb') as infile:
    word_embed = pickle.load(infile)
df

,artist,song,lyrics,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,language
0,b*witched feat. ladysmith black mambazo,i shall be there,awaken breathe hear call wind voice say journe...,0.276,0.670,2,-6.053,0.0322,0.4120,0.000033,0.1630,0.3290,89.642,4,en
1,b-2,y,small gang shit rest peace rest peace fuck bea...,0.653,0.493,8,-11.542,0.0370,0.1410,0.920000,0.0817,0.4320,111.005,4,en
2,b vocal,i want it that way,want way yeahyou fire one desire believe say w...,0.456,0.433,11,-5.450,0.0397,0.4730,0.000000,0.2520,0.6320,200.180,4,en
3,b vocal,more than words,say love word want hear want say know easy wou...,0.444,0.417,5,-7.850,0.0309,0.7450,0.000000,0.8690,0.3350,97.956,4,en
4,b*witched,rev it up,hurry ready ne bring gon na go slow take stead...,0.625,0.911,1,-6.629,0.0517,0.0545,0.000197,0.1860,0.7810,108.010,4,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75418,billy eckstine,if,make king would slave everything would still s...,0.317,0.217,4,-14.661,0.0499,0.9340,0.000860,0.2690,0.2590,78.650,4,en
75419,billy eckstine,jealousy,jealousy night day torture sometimes wonder sp...,0.356,0.321,10,-9.151,0.0383,0.9240,0.000000,0.2760,0.1980,112.148,4,en
75420,billy eckstine,i got lost in his arms,ask happen wish know believe happen still true...,0.178,0.288,10,-11.158,0.0318,0.9470,0.000076,0.3140,0.0897,174.971,3,en
75421,billy eckstine,i left my hat in haiti,leave hat haiti forget flat haiti could tell g...,0.423,0.265,3,-19.465,0.1080,0.9550,0.068700,0.3350,0.8190,120.193,4,en


In [3]:
X_train = word_embed[0:60000]
X_test = word_embed[60000:]

target_vars = ['liveness']
y = df[target_vars].to_numpy()
y_train = y[0:60000]
y_test = y[60000:]

(75423, 1024)

### Model building

In [54]:
def build_model():
    input_array = tf.keras.layers.Input(shape=(1024,), dtype='float')
    dense = tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(input_array)
    dense = tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(dense)
    out = tf.keras.layers.Dense(len(target_vars))(dense)
    model = tf.keras.Model(inputs=input_array, outputs=out)
    model.compile(loss='mean_absolute_error', optimizer='rmsprop', metrics=['mean_absolute_error'])
    return model
model = build_model()

In [55]:
model.summary()

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 1024)]            0         
_________________________________________________________________
dense_30 (Dense)             (None, 256)               262400    
_________________________________________________________________
dense_31 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_32 (Dense)             (None, 1)                 129       
Total params: 295,425
Trainable params: 295,425
Non-trainable params: 0
_________________________________________________________________


### Model Training

In [56]:
with tf.compat.v1.Session() as session:
    tf.compat.v1.keras.backend.set_session(session)
    session.run(tf.compat.v1.global_variables_initializer())
    session.run(tf.compat.v1.tables_initializer())
    history = model.fit(X_train, y_train, epochs=100,verbose=1, batch_size=1024, validation_split=0.2)
    model.save_weights('model_weights_{}_v1.h5'.format(target_vars[0]))

Train on 48000 samples, validate on 12000 samples
Epoch 1/100
48000/48000 [==============================] - 1s 10us/sample - loss: 0.6377 - mean_absolute_error: 0.2096 - val_loss: 0.4356 - val_mean_absolute_error: 0.1379
Epoch 2/100
48000/48000 [==============================] - 0s 7us/sample - loss: 0.3224 - mean_absolute_error: 0.1292 - val_loss: 0.2342 - val_mean_absolute_error: 0.1332
Epoch 3/100
48000/48000 [==============================] - 0s 7us/sample - loss: 0.1836 - mean_absolute_error: 0.1278 - val_loss: 0.1601 - val_mean_absolute_error: 0.1334
Epoch 4/100
48000/48000 [==============================] - 0s 7us/sample - loss: 0.1441 - mean_absolute_error: 0.1271 - val_loss: 0.1429 - val_mean_absolute_error: 0.1327
Epoch 5/100
48000/48000 [==============================] - 0s 7us/sample - loss: 0.1347 - mean_absolute_error: 0.1271 - val_loss: 0.1385 - val_mean_absolute_error: 0.1328
Epoch 6/100
48000/48000 [==============================] - 0s 7us/sample - loss: 0.1305 - mean

48000/48000 [==============================] - 0s 7us/sample - loss: 0.1250 - mean_absolute_error: 0.1247 - val_loss: 0.1325 - val_mean_absolute_error: 0.1322
Epoch 49/100
48000/48000 [==============================] - 0s 7us/sample - loss: 0.1249 - mean_absolute_error: 0.1246 - val_loss: 0.1325 - val_mean_absolute_error: 0.1322
Epoch 50/100
48000/48000 [==============================] - 0s 6us/sample - loss: 0.1250 - mean_absolute_error: 0.1247 - val_loss: 0.1325 - val_mean_absolute_error: 0.1322
Epoch 51/100
48000/48000 [==============================] - 0s 6us/sample - loss: 0.1250 - mean_absolute_error: 0.1247 - val_loss: 0.1330 - val_mean_absolute_error: 0.1327
Epoch 52/100
48000/48000 [==============================] - 0s 6us/sample - loss: 0.1250 - mean_absolute_error: 0.1247 - val_loss: 0.1325 - val_mean_absolute_error: 0.1322
Epoch 53/100
48000/48000 [==============================] - 0s 6us/sample - loss: 0.1250 - mean_absolute_error: 0.1247 - val_loss: 0.1325 - val_mean_abso

Epoch 95/100
48000/48000 [==============================] - 0s 6us/sample - loss: 0.1250 - mean_absolute_error: 0.1247 - val_loss: 0.1324 - val_mean_absolute_error: 0.1321
Epoch 96/100
48000/48000 [==============================] - 0s 6us/sample - loss: 0.1249 - mean_absolute_error: 0.1246 - val_loss: 0.1325 - val_mean_absolute_error: 0.1322
Epoch 97/100
48000/48000 [==============================] - 0s 6us/sample - loss: 0.1249 - mean_absolute_error: 0.1246 - val_loss: 0.1327 - val_mean_absolute_error: 0.1323
Epoch 98/100
48000/48000 [==============================] - 0s 6us/sample - loss: 0.1249 - mean_absolute_error: 0.1246 - val_loss: 0.1324 - val_mean_absolute_error: 0.1321
Epoch 99/100
48000/48000 [==============================] - 0s 6us/sample - loss: 0.1249 - mean_absolute_error: 0.1246 - val_loss: 0.1326 - val_mean_absolute_error: 0.1323
Epoch 100/100
48000/48000 [==============================] - 0s 6us/sample - loss: 0.1250 - mean_absolute_error: 0.1247 - val_loss: 0.1328 -

### Testing

In [49]:
model = build_model()

In [50]:
with tf.compat.v1.Session() as session:
    model.load_weights('./model_weights_{}_v1.h5'.format(target_vars[0]))
    loss, mae = model.evaluate(X_test,  y_test, verbose=2)
    print('Loss: {}, MAE: {}'.format(loss, mae))

15423/15423 - 1s - loss: 0.1065 - mean_absolute_error: 0.1054
Loss: 0.10649490759625924, MAE: 0.10541638731956482
